One-Hot-Encoding Step

In [12]:
#one hot encode
from numpy import array 
from numpy import argmax 
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
 
df = pd.read_csv(r"https://raw.githubusercontent.com/ECS-170/Heart-Disease-ML-Project/main/Raw_data/heart_2020.csv?token=GHSAT0AAAAAABT7CNQXGBIL6RF5DIUOUSDGYTJX4KQ")
print(df.head())
#Raw_data\heart_2020.csv

  HeartDisease    BMI Smoking AlcoholDrinking Stroke  PhysicalHealth  \
0           No  16.60     Yes              No     No             3.0   
1           No  20.34      No              No    Yes             0.0   
2           No  26.58     Yes              No     No            20.0   
3           No  24.21      No              No     No             0.0   
4           No  23.71      No              No     No            28.0   

   MentalHealth DiffWalking     Sex  AgeCategory   Race Diabetic  \
0          30.0          No  Female        55-59  White      Yes   
1           0.0          No  Female  80 or older  White       No   
2          30.0          No    Male        65-69  White      Yes   
3           0.0          No  Female        75-79  White       No   
4           0.0         Yes  Female        40-44  White       No   

  PhysicalActivity  GenHealth  SleepTime Asthma KidneyDisease SkinCancer  
0              Yes  Very good        5.0    Yes            No        Yes  
1       